# Goal

* Primer design for clade of interest

# Var

In [17]:
base_dir = '/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/'
clade = 'Anaerotruncus'
taxid = 244127 

# Init

In [18]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(LeyLabRMisc)

In [19]:
df.dims()
work_dir = file.path(base_dir, clade)
make_dir(work_dir)

Directory already exists: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted//Anaerotruncus 


# Genome download

* Downloading genomes from NCBI

```
OUTDIR=/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/
mkdir -p $OUTDIR
ncbi-genome-download -p 12 -s genbank -F fasta --genera Anaerotruncus -o $OUTDIR bacteria
```

# Genome quality

* Filtering genomes by quality

In [7]:
D = file.path(base_dir, clade, 'genbank')
files = list_files(D, '.fna.gz')
samps = data.frame(Name = files %>% as.character %>% basename,
                   Fasta = files,
                   Domain = 'Bacteria',
                   Taxid = taxid) %>%
    mutate(Name = gsub('\\.fna\\.gz$', '', Name),
           Fasta = gsub('/+', '/', Fasta))
samps

# writing file
outfile = file.path(D, 'samples.txt')
write_table(samps, outfile)

Name,Fasta,Domain,Taxid
<chr>,<chr>,<fct>,<dbl>
GCA_000154565.1_ASM15456v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/genbank/bacteria/GCA_000154565.1/GCA_000154565.1_ASM15456v1_genomic.fna.gz,Bacteria,244127
GCA_000403395.2_Anae_bact_G3_V1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/genbank/bacteria/GCA_000403395.2/GCA_000403395.2_Anae_bact_G3_V1_genomic.fna.gz,Bacteria,244127
⋮,⋮,⋮,⋮
GCA_902363605.1_MGYG-HGUT-00131_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/genbank/bacteria/GCA_902363605.1/GCA_902363605.1_MGYG-HGUT-00131_genomic.fna.gz,Bacteria,244127
GCA_904384175.1_Chicken_20_mag_154_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/genbank/bacteria/GCA_904384175.1/GCA_904384175.1_Chicken_20_mag_154_genomic.fna.gz,Bacteria,244127


File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted//Anaerotruncus/genbank/samples.txt 


### LLG

#### Config

In [8]:
cat_file(file.path(work_dir, 'config_llg.yaml'))

# table with genome --> fasta_file information
samples_file: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/genbank/samples.txt

# output location
output_dir: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/

# temporary file directory (your username will be added automatically)
tmp_dir: /ebio/abt3_scratch/

# batch processing of genomes for certain steps
## increase to better parallelize
batches: 2 

# Domain of genomes ('Archaea' or 'Bacteria)
## Use "Skip" if provided as a "Domain" column in the genome table
Domain: Skip

# software parameters
# Use "Skip" to skip any of these steps. If no params for rule, use ""
# dRep MAGs are not further analyzed, but you can de-rep & then use the de-rep genome table as input.
params:
  ionice: -c 3
  # assembly assessment
  seqkit: ""
  quast: Skip #""
  multiqc_on_quast: "" 
  checkm: ""
  # de-replication (requires checkm)
  drep: -com

#### Run

```
(snakemake) @ rick:/ebio/abt3_projects/software/dev/ll_pipelines/llg
$ screen -L -S llg ./snakemake_sge.sh /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/config_llg.yaml 20 -F
```

### Samples table of high-quality genomes

In [20]:
# checkM summary
checkm = file.path(work_dir, 'LLG_output', 'checkM', 'checkm_qa_summary.tsv') %>%
    read.delim(sep='\t') 
checkm

Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,Strain.heterogeneity,Genome.size..bp.,X..ambiguous.bases,⋯,X0,X1,X2,X3,X4,X5.,assembly.Id,assembler.Id,taxon.Id,File
<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<fct>,<fct>,<lgl>,<fct>
GCA_000154565.1_ASM15456v1_genomic,o__Clostridiales (UID1212),172,263,149,97.99,0.22,0,3719688,800,⋯,3,259,1,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Anaerotruncus|LLG_output|checkM|1|checkm|markers_qa_summary.tsv.1,markers_qa_summary.tsv.1,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/checkM/1/checkm/markers_qa_summary.tsv.1
GCA_000433975.1_MGS528_genomic,o__Clostridiales (UID1212),172,263,149,97.99,0.00,0,1881083,732,⋯,3,260,0,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Anaerotruncus|LLG_output|checkM|1|checkm|markers_qa_summary.tsv.2,markers_qa_summary.tsv.2,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/checkM/1/checkm/markers_qa_summary.tsv.2
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GCA_900199635.1_PRJEB19460_genomic,o__Clostridiales (UID1212),172,263,149,99.33,0.00,0,3145951,14,⋯,1,262,0,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Anaerotruncus|LLG_output|checkM|2|checkm|markers_qa_summary.tsv.14,markers_qa_summary.tsv.14,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/checkM/2/checkm/markers_qa_summary.tsv.14
GCA_902363605.1_MGYG-HGUT-00131_genomic,o__Clostridiales (UID1212),172,263,149,97.99,1.34,0,3306065,265,⋯,3,257,3,0,0,0,|ebio|abt3_projects|software|dev|ll_pipelines|llprimer|experiments|HMP_most-wanted|Anaerotruncus|LLG_output|checkM|2|checkm|markers_qa_summary.tsv.15,markers_qa_summary.tsv.15,NA,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/checkM/2/checkm/markers_qa_summary.tsv.15


In [21]:
# dRep summary
drep = file.path(work_dir, 'LLG_output', 'drep', 'checkm_markers_qa_summary.tsv') %>%
    read.delim(sep='\t') %>%
    mutate(Bin.Id = gsub('.+/', '', genome),
           Bin.Id = gsub('\\.fna$', '', Bin.Id))
drep

genome,completeness,contamination,Bin.Id
<fct>,<dbl>,<dbl>,<chr>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000154565.1_ASM15456v1_genomic.fna,97.99,0.22,GCA_000154565.1_ASM15456v1_genomic
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000433975.1_MGS528_genomic.fna,97.99,0.00,GCA_000433975.1_MGS528_genomic
⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900199635.1_PRJEB19460_genomic.fna,99.33,0.00,GCA_900199635.1_PRJEB19460_genomic
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_902363605.1_MGYG-HGUT-00131_genomic.fna,97.99,1.34,GCA_902363605.1_MGYG-HGUT-00131_genomic


In [22]:
# de-replicated genomes
drep_gen = file.path(work_dir, 'LLG_output', 'drep', 'dereplicated_genomes.tsv') %>%
    read.delim(sep='\t')
drep_gen

Name,Fasta
<fct>,<fct>
GCA_014284405.1_ASM1428440v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/drep/drep/dereplicated_genomes/GCA_014284405.1_ASM1428440v1_genomic.fna
GCA_900199635.1_PRJEB19460_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/drep/drep/dereplicated_genomes/GCA_900199635.1_PRJEB19460_genomic.fna
⋮,⋮
GCA_015554285.1_ASM1555428v1_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/drep/drep/dereplicated_genomes/GCA_015554285.1_ASM1555428v1_genomic.fna
GCA_000433975.1_MGS528_genomic,/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/LLG_output/drep/drep/dereplicated_genomes/GCA_000433975.1_MGS528_genomic.fna


In [23]:
# GTDBTk summary
tax = file.path(work_dir, 'LLG_output', 'gtdbtk', 'gtdbtk_bac_summary.tsv') %>%
    read.delim(, sep='\t') %>%
    separate(classification, 
             c('Domain', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species'),
             sep=';') %>%
    select(-note, -classification_method, -pplacer_taxonomy,
           -other_related_references.genome_id.species_name.radius.ANI.AF.)
tax

user_genome,Domain,Phylum,Class,Order,Family,Genus,Species,fastani_reference,fastani_reference_radius,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
GCA_000154565.1_ASM15456v1_genomic,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Ruminococcaceae,g__Anaerotruncus,s__Anaerotruncus colihominis,GCF_000154565.1,95.0,⋯,1.0,GCF_000154565.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus colihominis,100.0,1.0,97.18,11,N/A,N/A
GCA_000433975.1_MGS528_genomic,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Acutalibacteraceae,g__Eubacterium_R,s__Eubacterium_R sp000433975,GCA_000433975.1,95.0,⋯,1.0,GCA_000433975.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Acutalibacteraceae;g__Eubacterium_R;s__Eubacterium_R sp000433975,100.0,1.0,96.57,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
GCA_900199635.1_PRJEB19460_genomic,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Ruminococcaceae,g__Anaerotruncus,s__Anaerotruncus massiliensis,GCF_900199635.1,95.0,⋯,1.0,GCF_900199635.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus massiliensis,100.0,1.0,96.43,11,N/A,N/A
GCA_902363605.1_MGYG-HGUT-00131_genomic,d__Bacteria,p__Firmicutes_A,c__Clostridia,o__Oscillospirales,f__Ruminococcaceae,g__Anaerotruncus,s__Anaerotruncus rubiinfantis,GCF_900078395.1,95.0,⋯,0.91,GCF_900078395.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus rubiinfantis,99.26,0.91,96.35,11,N/A,N/A


In [24]:
# checking overlap
cat('-- drep --\n')
overlap(basename(as.character(drep_gen$Fasta)), 
        basename(as.character(drep$genome)))
cat('-- checkm --\n')
overlap(drep$Bin.Id, checkm$Bin.Id)
cat('-- gtdbtk --\n')
overlap(drep$Bin.Id, tax$user_genome)

-- drep --
intersect(x,y): 27 
setdiff(x,y): 0 
setdiff(y,x): 4 
union(x,y): 31 
-- checkm --
intersect(x,y): 31 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 31 
-- gtdbtk --
intersect(x,y): 30 
setdiff(x,y): 1 
setdiff(y,x): 0 
union(x,y): 31 


In [25]:
# joining based on Bin.Id
drep = drep %>%
    inner_join(checkm, c('Bin.Id')) %>%
    mutate(GEN = genome %>% as.character %>% basename) %>%
    inner_join(drep_gen %>% mutate(GEN = Fasta %>% as.character %>% basename),
               by=c('GEN')) %>%
    inner_join(tax, c('Bin.Id'='user_genome')) #%>%
drep

genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000154565.1_ASM15456v1_genomic.fna,97.99,0.22,GCA_000154565.1_ASM15456v1_genomic,o__Clostridiales (UID1212),172,263,149,97.99,0.22,⋯,1.0,GCF_000154565.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus colihominis,100.0,1.0,97.18,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000433975.1_MGS528_genomic.fna,97.99,0.00,GCA_000433975.1_MGS528_genomic,o__Clostridiales (UID1212),172,263,149,97.99,0.00,⋯,1.0,GCA_000433975.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Acutalibacteraceae;g__Eubacterium_R;s__Eubacterium_R sp000433975,100.0,1.0,96.57,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900199635.1_PRJEB19460_genomic.fna,99.33,0.00,GCA_900199635.1_PRJEB19460_genomic,o__Clostridiales (UID1212),172,263,149,99.33,0.00,⋯,1.0,GCF_900199635.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus massiliensis,100.0,1.0,96.43,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_902363605.1_MGYG-HGUT-00131_genomic.fna,97.99,1.34,GCA_902363605.1_MGYG-HGUT-00131_genomic,o__Clostridiales (UID1212),172,263,149,97.99,1.34,⋯,0.91,GCF_900078395.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus rubiinfantis,99.26,0.91,96.35,11,N/A,N/A


In [38]:
# filtering by quality
hq_genomes = drep %>%
    filter(completeness >= 90,
           contamination < 5,
           Strain.heterogeneity < 50,
           X..contigs < 300,
           Mean.contig.length..bp. > 10000)
hq_genomes

genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000154565.1_ASM15456v1_genomic.fna,97.99,0.22,GCA_000154565.1_ASM15456v1_genomic,o__Clostridiales (UID1212),172,263,149,97.99,0.22,⋯,1.0,GCF_000154565.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus colihominis,100.0,1.0,97.18,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000433975.1_MGS528_genomic.fna,97.99,0.00,GCA_000433975.1_MGS528_genomic,o__Clostridiales (UID1212),172,263,149,97.99,0.00,⋯,1.0,GCA_000433975.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Acutalibacteraceae;g__Eubacterium_R;s__Eubacterium_R sp000433975,100.0,1.0,96.57,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900199635.1_PRJEB19460_genomic.fna,99.33,0.00,GCA_900199635.1_PRJEB19460_genomic,o__Clostridiales (UID1212),172,263,149,99.33,0.00,⋯,1.0,GCF_900199635.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus massiliensis,100.0,1.0,96.43,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_902363605.1_MGYG-HGUT-00131_genomic.fna,97.99,1.34,GCA_902363605.1_MGYG-HGUT-00131_genomic,o__Clostridiales (UID1212),172,263,149,97.99,1.34,⋯,0.91,GCF_900078395.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus rubiinfantis,99.26,0.91,96.35,11,N/A,N/A


In [39]:
# summarizing the genome stats
hq_genomes %>% summary_x('completeness', completeness)
hq_genomes %>% summary_x('contamination', contamination)
hq_genomes %>% summary_x('No. of contigs', X..contigs)
hq_genomes %>% summary_x('N50', N50..contigs.)
hq_genomes %>% summary_x('Mean contig length', Mean.contig.length..bp.)

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
completeness,97.32,97.99,97.99,98.24538,98.66,99.33,0.683,0.279


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
contamination,0,0.055,0.22,0.4280769,0.59,1.34,0.495,0.202


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
No. of contigs,8,36,68,72.15385,96,251,85.307,34.826


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
N50,48108,87859.5,112468.5,303948.2,248682.5,2375707,909825.5,371434.7


,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
Mean contig length,14550,37144.25,45687,87416.5,101514.8,393242,140924.9,57532.35


In [40]:
# summarizing the taxonomy
df.dims(20)
hq_genomes %>%
    group_by(Family, Genus) %>%
    summarize(n_genomes = n(), .groups='drop')
df.dims()

Family,Genus,n_genomes
<chr>,<chr>,<int>
f__Acutalibacteraceae,g__Eubacterium_R,1
f__Anaerovoracaceae,g__Emergencia,1
f__Ruminococcaceae,g__Anaerotruncus,24


In [43]:
# filtering by taxonomy
hq_genomes = hq_genomes %>%
    filter(Genus == 'g__Anaerotruncus')
hq_genomes

genome,completeness,contamination,Bin.Id,Marker.lineage,X..genomes,X..markers,X..marker.sets,Completeness,Contamination,⋯,fastani_af,closest_placement_reference,closest_placement_radius,closest_placement_taxonomy,closest_placement_ani,closest_placement_af,msa_percent,translation_table,red_value,warnings
<fct>,<dbl>,<dbl>,<chr>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<dbl>,<int>,<fct>,<fct>
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_000154565.1_ASM15456v1_genomic.fna,97.99,0.22,GCA_000154565.1_ASM15456v1_genomic,o__Clostridiales (UID1212),172,263,149,97.99,0.22,⋯,1.0,GCF_000154565.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus colihominis,100.0,1.0,97.18,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_001404495.1_14207_7_62_genomic.fna,97.99,0.36,GCA_001404495.1_14207_7_62_genomic,o__Clostridiales (UID1212),172,263,149,97.99,0.36,⋯,0.83,GCF_000154565.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus colihominis,99.07,0.83,95.50,11,N/A,N/A
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_900199635.1_PRJEB19460_genomic.fna,99.33,0.00,GCA_900199635.1_PRJEB19460_genomic,o__Clostridiales (UID1212),172,263,149,99.33,0.00,⋯,1.0,GCF_900199635.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus massiliensis,100.0,1.0,96.43,11,N/A,N/A
/ebio/abt3_scratch/nyoungblut/LLG_62325884640/genomes/GCA_902363605.1_MGYG-HGUT-00131_genomic.fna,97.99,1.34,GCA_902363605.1_MGYG-HGUT-00131_genomic,o__Clostridiales (UID1212),172,263,149,97.99,1.34,⋯,0.91,GCF_900078395.1,95.0,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__Oscillospirales;f__Ruminococcaceae;g__Anaerotruncus;s__Anaerotruncus rubiinfantis,99.26,0.91,96.35,11,N/A,N/A


In [44]:
# writing samples table for LLPRIMER
outfile = file.path(work_dir, 'samples_genomes_hq.txt')
hq_genomes %>%
    select(Bin.Id, Fasta) %>%
    rename('Taxon' = Bin.Id) %>%
    mutate(Taxon = gsub('_chromosome.+', '', Taxon),
           Taxon = gsub('_bin_.+', '', Taxon),
           Taxon = gsub('_genomic', '', Taxon),
           Taxon = gsub('_annotated_assembly', '', Taxon),
           Taxid = taxid) %>%
    write_table(outfile)

File written: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted//Anaerotruncus/samples_genomes_hq.txt 


# Primer design

### Config

In [42]:
F = file.path(work_dir, 'primers', 'config.yaml')
cat_file(F)

#-- I/O --#
samples_file: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/samples_genomes_hq.txt

# output location
output_dir: /ebio/abt3_projects/software/dev/ll_pipelines/llprimer/experiments/HMP_most-wanted/Anaerotruncus/primers/

# temporary file directory (your username will be added automatically)
tmp_dir: /ebio/abt3_scratch/

#-- software parameters --#
# See the README for a description
params:
  ionice: -c 3
  cgp:
    prodigal: ""    
    mmseqs:
      method: cluster    # or linclust (faster)
      run: --min-seq-id 0.8 -c 0.8
    core_genes: --frac 1 --max-clusters 500
    blastx: -evalue 1e-10 -max_target_seqs 3
    blastx_nontarget: -evalue 1e-5 -max_target_seqs 30
    align:
      method: linsi
      params: --auto --maxiterate 1000
    primer3:
      number: --num-primers 500
      size: --opt-size 20 --min-size 18 --max-size 24
      product: --opt-prod-size 150 --min-prod-size 100 --max-prod-size 200
      Tm: --opt-tm

### Run

```
(snakemake) @ rick:/ebio/abt3_projects/software/dev/ll_pipelines/llprimer
$ screen -L -S llprimer-aner ./snakemake_sge.sh experiments/HMP_most-wanted/Anaerotruncus/primers/config.yaml 50 --notemp -F
```

### Summary

In [52]:
primer_info = read.delim(file.path(work_dir, 'primers', 'cgp', 'primers_final_info.tsv'), sep='\t')
primer_info %>% unique_n('primer sets', primer_set)
primer_info

No. of unique primer sets: 276 


cluster_id,primer_set,amplicon_size_consensus,amplicon_size_avg,amplicon_size_sd,primer_id,primer_type,sequence,length,degeneracy,⋯,position_start,position_end,Tm_avg,Tm_sd,GC_avg,GC_sd,hairpin_avg,hairpin_sd,homodimer_avg,homodimer_sd
<int>,<int>,<int>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<int>,<int>,⋯,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4,267,121,121,0,267f,PRIMER_LEFT,KMSAAGCTGCTSAARTCGGC,20,32,⋯,141,161,62.20092,1.817715,57.50000,4.330127,44.12935,7.786871,6.123101,9.199705
4,267,121,121,0,267r,PRIMER_RIGHT,GGGGRCGRATVCGYTTCAT,19,24,⋯,243,262,61.25268,2.574313,58.77193,5.189544,53.52857,12.395231,7.186602,22.559500
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
246,438,165,165,0,438f,PRIMER_LEFT,YGAYTTYCCGCTBTCMAACTG,21,48,⋯,122,143,60.35574,1.733370,50.79365,5.264484,4.452419,11.83836,-38.53541,5.906476
246,438,165,165,0,438r,PRIMER_RIGHT,GAGSWYTTCTGRTAVGGCGG,20,48,⋯,267,287,60.29829,1.778744,58.33333,4.249183,13.355395,18.42953,-13.88619,23.840250


### Gene cluster annotations

In [46]:
gene_annot = read.delim(file.path(work_dir, 'primers', 'cgp', 'core_clusters_blastx.tsv'), 
                        sep='\t') %>%
    mutate(cluster_id = gsub('cluster_', '', cluster_id) %>% as.Num) %>%
    semi_join(primer_info, c('cluster_id')) %>%
    mutate(gene_name = gsub(' \\[.+', '', subject_name),
           gene_taxonomy = gsub('.+\\[', '', subject_name),
           gene_taxonomy = gsub('\\]', '', gene_taxonomy))
gene_annot

cluster_id,query,subject,subject_name,pident,length,mismatch,qstart,qend,sstart,send,evalue,slen,qlen,sscinames,staxids,pident_rank,gene_name,gene_taxonomy
<dbl>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<fct>,<fct>,<int>,<chr>,<chr>
4,abf8b17f07404c7183f7f0f8c044a48f,WP_066455364.1,MULTISPECIES: 50S ribosomal protein L22 [Anaerotruncus],100.000,111,0,1,333,1,111,3.92e-75,111,336,Anaerotruncus;Anaerotruncus rubiinfantis;Anaerotruncus sp. AF02-27,244127;1720200;2292191,3,MULTISPECIES: 50S ribosomal protein L22,Anaerotruncus
4,abf8b17f07404c7183f7f0f8c044a48f,WP_147539073.1,50S ribosomal protein L22 [Anaerotruncus rubiinfantis],99.099,111,1,1,333,1,111,1.26e-74,111,336,Anaerotruncus rubiinfantis,1720200,1,50S ribosomal protein L22,Anaerotruncus rubiinfantis
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
246,ce2e4df7569647278c025e0d6334cd91,WP_006874432.1,glucose-1-phosphate adenylyltransferase [Anaerotruncus colihominis],99.762,420,1,1,1260,1,420,0,420,1263,Anaerotruncus colihominis;Anaerotruncus colihominis DSM 17241,169435;445972,1,glucose-1-phosphate adenylyltransferase,Anaerotruncus colihominis
246,ce2e4df7569647278c025e0d6334cd91,WP_195462589.1,glucose-1-phosphate adenylyltransferase [Anaerotruncus colihominis],99.048,420,4,1,1260,1,420,0,420,1263,Anaerotruncus colihominis,169435,2,glucose-1-phosphate adenylyltransferase,Anaerotruncus colihominis


In [47]:
df.dims(50)
gene_annot %>%
    distinct(cluster_id, gene_name) 
df.dims()

cluster_id,gene_name
<dbl>,<chr>
4,MULTISPECIES: 50S ribosomal protein L22
4,50S ribosomal protein L22
5,MULTISPECIES: type I DNA topoisomerase
5,type I DNA topoisomerase
10,Catabolite repression HPr
10,HPr family phosphocarrier protein
10,MULTISPECIES: HPr family phosphocarrier protein
11,50S ribosomal protein L30
11,MULTISPECIES: 50S ribosomal protein L30


In [48]:
df.dims(50)
gene_annot %>%
    distinct(cluster_id, gene_taxonomy) 
df.dims()

cluster_id,gene_taxonomy
<dbl>,<chr>
4,Anaerotruncus
4,Anaerotruncus rubiinfantis
4,unclassified Anaerotruncus
5,Anaerotruncus
5,Anaerotruncus rubiinfantis
5,Anaerotruncus sp. G3(2012)
10,Anaerotruncus colihominis
10,Anaerotruncus
11,Anaerotruncus colihominis


### Gene cluster: closest related

In [49]:
gene_annot = read.delim(file.path(work_dir, 'primers', 'cgp', 'core_clusters_blastx_nontarget.tsv'), 
                        sep='\t') %>%
    mutate(cluster_id = gsub('cluster_', '', cluster_id) %>% as.Num) %>%
    semi_join(primer_info, c('cluster_id')) %>%
    mutate(gene_name = gsub(' \\[.+', '', subject_name),
           gene_taxonomy = gsub('.+\\[', '', subject_name),
           gene_taxonomy = gsub('\\]', '', gene_taxonomy))
gene_annot

cluster_id,query,subject,subject_name,pident,length,mismatch,qstart,qend,sstart,send,evalue,slen,qlen,sscinames,staxids,pident_rank,gene_name,gene_taxonomy
<dbl>,<fct>,<fct>,<fct>,<dbl>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<fct>,<fct>,<int>,<chr>,<chr>
4,abf8b17f07404c7183f7f0f8c044a48f,WP_101908516.1,50S ribosomal protein L22 [Marasmitruncus massiliensis],91.892,111,9,1,333,1,111,1.45e-70,111,336,Ruminococcaceae bacterium;Marasmitruncus massiliensis,1898205;1944642,1,50S ribosomal protein L22,Marasmitruncus massiliensis
4,abf8b17f07404c7183f7f0f8c044a48f,MBC8585943.1,50S ribosomal protein L22 [Oscillospiraceae bacterium NSJ-64],90.090,111,11,1,333,1,111,1.60e-69,111,336,Oscillospiraceae bacterium NSJ-64,2763678,2,50S ribosomal protein L22,Oscillospiraceae bacterium NSJ-64
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
246,ce2e4df7569647278c025e0d6334cd91,WP_090266788.1,glucose-1-phosphate adenylyltransferase [Anaeromassilibacillus sp. Marseille-P4683],73.580,405,107,1,1215,1,405,0,405,1263,Eubacteriaceae bacterium CHKCI005;Anaeromassilibacillus sp. Marseille-P4683,1780381;2041843,28,glucose-1-phosphate adenylyltransferase,Anaeromassilibacillus sp. Marseille-P4683
246,ce2e4df7569647278c025e0d6334cd91,MBE6902255.1,glucose-1-phosphate adenylyltransferase [Ruminococcaceae bacterium],73.048,397,107,10,1200,5,401,0,401,1263,Ruminococcaceae bacterium,1898205,30,glucose-1-phosphate adenylyltransferase,Ruminococcaceae bacterium


In [50]:
df.dims(50)
gene_annot %>%
    filter(pident > 80,
           pident_rank <= 3) %>%
    select(cluster_id, gene_name, gene_taxonomy, pident)
    
df.dims()

cluster_id,gene_name,gene_taxonomy,pident
<dbl>,<chr>,<chr>,<dbl>
4,50S ribosomal protein L22,Marasmitruncus massiliensis,91.892
4,50S ribosomal protein L22,Oscillospiraceae bacterium NSJ-64,90.090
4,50S ribosomal protein L22,bacterium 1XD42-8,89.189
5,type I DNA topoisomerase,Ruminococcaceae bacterium,84.173
5,type I DNA topoisomerase,Marasmitruncus massiliensis,84.029
5,type I DNA topoisomerase,Oscillospiraceae bacterium NSJ-64,81.727
10,HPr family phosphocarrier protein,Marasmitruncus massiliensis,97.701
10,HPr family phosphocarrier protein,bacterium 1XD42-8,97.701
10,HPr family phosphocarrier protein,Hydrogenoanaerobacterium saccharovorans,95.402


In [51]:
df.dims(50)
gene_annot %>%
    distinct(cluster_id, gene_name) 
df.dims()

cluster_id,gene_name
<dbl>,<chr>
4,50S ribosomal protein L22
4,MULTISPECIES: 50S ribosomal protein L22
5,type I DNA topoisomerase
5,MULTISPECIES: type I DNA topoisomerase
5,unnamed protein product
5,dNA topoisomerase
5,DNA topoisomerase-1
5,DNA topoisomerase 1
10,HPr family phosphocarrier protein


# sessionInfo

In [18]:
pipelineInfo('/ebio/abt3_projects/software/dev/ll_pipelines/llg/')

LLG
===

Ley Lab Genome analysis pipeline (LLG)

* Version: 0.1.9
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/gtdbtk.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- pigz
- bioconda::gtdbtk

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/checkm.yaml <==
channels:
- bioconda
dependencies:
- python=2.7
- pigz
- bioconda::prodigal
- bioconda::pplacer
- bioconda::checkm-genome

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/quast.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- bioconda::seqkit
- bioconda::quast>=5.0.0

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/sourmash.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- bioconda::sourmash=2.0.0a4

==> /ebio/abt3_projects/software/dev/ll_pipelines/llg//bin/envs/fastqc.yaml <==
channels:
- conda-forge
- bioconda


In [19]:
pipelineInfo('/ebio/abt3_projects/software/dev/ll_pipelines/llprimer/')

LLPRIMER

Ley Lab Primer generation pipeline (LLPRIMER)

* Version: 0.2.2
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/pdp.yaml <==
channels:
- conda-forge
- bioconda
dependencies:
- python=3.7
- intervaltree
- prodigal
- blast
- bedtools
- mafft
- mummer=3.23
- emboss
- primer3=1.1.4
- biopython<1.78
- pybedtools
- joblib
- tqdm
- openpyxl

==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/genes.yaml <==
channels:
- bioconda
dependencies:
- pigz
- python=3
- numpy
- pyfaidx
- bioconda::seqkit
- bioconda::fasta-splitter
- bioconda::vsearch
- bioconda::prodigal
- bioconda::mmseqs2
==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/envs/aln.yaml <==
channels:
- bioconda
- conda-forge
dependencies:
- pigz
- bioconda::kalign3
- bioconda::mafft

==> /ebio/abt3_projects/software/dev/ll_pipelines/llprimer//bin/env

In [20]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/tidyverse/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] LeyLabRMisc_0.1.6 ggplot2_3.3.1     tidyr_1.1.0       dplyr_1.0.0      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.4.6     magrittr_1.5     munsell_0.5.0    tidyselect_1.1.0
 [5] uuid_0.1-4       colorspace_1.4-1 R6_2.4.1         rlang_0.4.6     
 [9] tools_3.6.3      grid_3.6.3       gtable_0.3.0     withr_2.2.0     
[13] htmltools_0.4.0  ellipsis_0.3.1 